In [1]:
!pip install dash pandas plotly ucimlrepo
!pip install dash-bootstrap-components

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.7/203.7 kB 6.3 MB/s eta 0:00:00


In [2]:
from dash import Dash, dcc, html, Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd
from ucimlrepo import fetch_ucirepo
from sklearn.tree import DecisionTreeRegressor
import numpy as np

wine = fetch_ucirepo(id=186)
df = wine.data.features.copy()
df['quality'] = wine.data.targets['quality']

nombre_legible = {
    "fixed_acidity": "Acidez fija",
    "volatile_acidity": "Acidez volátil",
    "citric_acid": "Ácido cítrico",
    "residual_sugar": "Azúcar residual",
    "chlorides": "Cloruros",
    "free_sulfur_dioxide": "Dióxido de azufre libre",
    "total_sulfur_dioxide": "Dióxido de azufre total",
    "density": "Densidad",
    "pH": "pH",
    "sulphates": "Sulfatos",
    "alcohol": "Alcohol",
    "quality": "Calidad del vino"
}

descripcion_variables = {
    "Acidez fija": "Cantidad de ácidos no volátiles en el vino (como el tartárico)",
    "Acidez volátil": "Ácido acético, relacionado con sabores desagradables",
    "Ácido cítrico": "Preservante natural, mejora sabor y frescura",
    "Azúcar residual": "Azúcar no fermentada que permanece en el vino",
    "Cloruros": "Contenido de sal (influye en el sabor)",
    "Dióxido de azufre libre": "Protección antimicrobiana activa",
    "Dióxido de azufre total": "Suma de azufre libre y combinado (preservación)",
    "Densidad": "Relación entre masa y volumen (conectado con azúcar)",
    "pH": "Acidez o basicidad del vino",
    "Sulfatos": "Conservante que mejora estabilidad microbiológica",
    "Alcohol": "Porcentaje de alcohol en volumen",
    "Calidad del vino": "Evaluación sensorial del vino (0 a 10)"
}

nombre_inverso = {v: k for k, v in nombre_legible.items()}
correlation_matrix = df.corr(numeric_only=True)

app = Dash(__name__, external_stylesheets=[dbc.themes.LUX])
app.title = "Dashboard Calidad del Vino"

style_card = {
    'backgroundColor': 'white',
    'padding': '20px',
    'borderRadius': '10px',
    'boxShadow': '0px 2px 8px rgba(0,0,0,0.1)',
    'marginBottom': '20px',
    'height': '100%'
}

def texto_correlacion_mejorado(corr, variable):
    fuerza = "fuerte" if abs(corr) > 0.6 else "moderada" if abs(corr) > 0.3 else "débil"
    tipo = "positiva" if corr > 0 else "negativa"
    return f"Correlación {tipo} {fuerza} entre **{nombre_legible[variable]}** y **Calidad del vino**: **{corr:.2f}**"


app.layout = dbc.Container([
    html.H1("Dashboard: Análisis de Calidad del Vino", style={"textAlign": "center", "marginTop": "20px", "marginBottom": "30px", 'fontFamily':'Segoe UI, sans-serif'}),

    dbc.Tabs([

        dbc.Tab(label="Resumen", children=[
            dbc.Row([
                dbc.Col(dbc.Card([
                    html.H5("Calidad Promedio", className="card-title", style={'textAlign': 'center'}),
                    html.H2(f"{df['quality'].mean():.2f}", style={'textAlign': 'center', 'color':'#8B0000'})
                ], style=style_card), width=3),
                dbc.Col(dbc.Card([
                    html.H5("Alcohol Promedio", className="card-title", style={'textAlign': 'center'}),
                    html.H2(f"{df['alcohol'].mean():.2f}", style={'textAlign': 'center', 'color':'#8B0000'})
                ], style=style_card), width=3),
                dbc.Col(dbc.Card([
                    html.H5("Variable más correlacionada", className="card-title", style={'textAlign': 'center'}),

                    html.H2(
                        f"{nombre_legible[correlation_matrix['quality'].drop('quality').abs().idxmax()]}",
                        style={'textAlign': 'center', 'color':'#8B0000'}
                    )
                ], style=style_card), width=3)
            ], justify="center", className="mb-4"),

            dbc.Card([
                html.H4("Distribución General de la Calidad del Vino", className="card-title", style={"textAlign": "center"}),
                dcc.Graph(
                    figure=px.pie(df, names='quality', title='Distribución de la Calidad del Vino',
                                  color_discrete_sequence=px.colors.sequential.RdBu)
                )
            ], style=style_card)
        ], style={'padding': '20px'}),

        dbc.Tab(label="Análisis Variable", children=[
            dbc.Card([
                html.H4("Significado de cada variable", style={"textAlign": "center", "marginBottom": "20px"}),
                dbc.Table(
                    [html.Thead(html.Tr([html.Th("Variable"), html.Th("Descripción")]))] +
                    [html.Tr([html.Td(var), html.Td(desc)]) for var, desc in descripcion_variables.items()],
                    bordered=True, hover=True, responsive=True, striped=True,
                    style={'maxHeight': '300px', 'overflowY': 'auto'}
                ),
            ], style=style_card),

            html.Br(),

            dbc.Row([
                dbc.Col([
                    html.Label("Selecciona una variable:", style={'fontWeight': 'bold'}),
                    dcc.Dropdown(
                        id='variable',
                        options=[{'label': nombre_legible[col], 'value': col} for col in df.columns if col != 'quality'],
                        value='alcohol',
                        clearable=False
                    ),
                    html.Br(),
                    dcc.RadioItems(
                        id='tipo_grafico',
                        options=[
                            {'label': 'Histograma (promedio calidad)', 'value': 'hist'},
                            {'label': 'Gráfico de dispersión', 'value': 'scatter'}
                        ],
                        value='hist',
                        inline=True
                    ),
                    html.Br(),
                    html.Div(id='texto_correlacion', style={"fontSize": "18px", "textAlign": "center", "marginTop": "15px"}),
                ], width=6),
                dbc.Col(dcc.Graph(id='grafico_variable'), width=6)
            ]),

            html.Hr(),

            dbc.Card([
                html.H4("Importancia de las variables en la calidad del vino", style={"textAlign": "center"}),
                dcc.Graph(id='grafica_importancia_pastel')
            ], style=style_card)
        ], style={'padding': '20px'}),

        dbc.Tab(label="Correlaciones", children=[
            dbc.Card([
                html.H4("Mapa de Correlación", style={"textAlign": "center", "marginBottom": "10px"}),
                html.P("Selecciona una variable para resaltar sus correlaciones:", style={'textAlign': 'center'}),
                dcc.Dropdown(
                    id='variable_selector',
                    options=[{'label': nombre_legible[var], 'value': var} for var in correlation_matrix.columns],
                    value='quality',
                    clearable=False,
                    style={'width': '50%', 'margin': 'auto'}
                ),
                dcc.Graph(id='correlation_heatmap')
            ], style=style_card)
        ], style={'padding': '20px'}),
    ])
], fluid=True, style={'backgroundColor': '#F8F9FA', 'minHeight': '100vh', 'fontFamily':'Segoe UI, sans-serif'})


@app.callback(
    Output('grafico_variable', 'figure'),
    Output('texto_correlacion', 'children'),
    Input('variable', 'value'),
    Input('tipo_grafico', 'value')
)
def actualizar_grafico(variable, tipo_grafico):
    corr = df[variable].corr(df['quality'])
    texto = texto_correlacion_mejorado(corr, variable)

    if tipo_grafico == 'hist':
        fig = px.histogram(
            df, x=variable, y='quality', nbins=20,
            title=f'Calidad del vino según {nombre_legible[variable]}',
            labels={'quality': 'Calidad del vino', variable: nombre_legible[variable]},
            histfunc='avg',
            color_discrete_sequence=['#8B0000']
        )
        fig.update_layout(bargap=0.1)
    else:
        fig = px.scatter(
            df, x=variable, y='quality',
            title=f'Dispersión de calidad vs {nombre_legible[variable]}',
            labels={'quality': 'Calidad del vino', variable: nombre_legible[variable]},
            color_discrete_sequence=['#8B0000']
        )
        fig.update_traces(marker=dict(size=8, opacity=0.6))

    fig.update_layout(title_x=0.5)
    return fig, texto


@app.callback(
    Output('grafica_importancia_pastel', 'figure'),
    Input('variable', 'value')
)
def calcular_importancia(_):
    X = df.drop(columns=['quality'])
    y = df['quality']

    modelo = DecisionTreeRegressor(random_state=42)
    modelo.fit(X, y)

    importancias = modelo.feature_importances_
    variables = X.columns

    df_importancias = pd.DataFrame({
        'Variable': variables,
        'Importancia': importancias
    }).sort_values(by='Importancia', ascending=False)

    df_importancias['Variable'] = df_importancias['Variable'].map(nombre_legible)
    df_importancias['Porcentaje'] = df_importancias['Importancia'] * 100

    fig = px.pie(df_importancias, names='Variable', values='Porcentaje',
                 title="Importancia de las variables en la calidad del vino",
                 labels={'Porcentaje': 'Importancia (%)'},
                 color_discrete_sequence=px.colors.sequential.RdBu)
    fig.update_layout(title_x=0.5)

    return fig


@app.callback(
    Output('correlation_heatmap', 'figure'),
    Input('variable_selector', 'value')
)
def actualizar_heatmap(variable_destacada):

    opacity_matrix = np.full(correlation_matrix.shape, 0.4)
    col_idx = correlation_matrix.columns.get_loc(variable_destacada)
    opacity_matrix[:, col_idx] = 1
    opacity_matrix[col_idx, :] = 1

    fig = px.imshow(
        correlation_matrix,
        text_auto=".2f",
        color_continuous_scale='RdBu_r',
        title=f'Matriz de Correlación - Variable Seleccionada: {nombre_legible[variable_destacada]}',
        aspect='auto'
    )

    fig.update_traces(zmin=-1, zmax=1)

    fig.add_shape(
        type="rect",
        x0=col_idx - 0.5, x1=col_idx + 0.5,
        y0=-0.5, y1=correlation_matrix.shape[0] - 0.5,
        line=dict(color="black", width=3)
    )
    fig.add_shape(
        type="rect",
        x0=-0.5, x1=correlation_matrix.shape[1] - 0.5,
        y0=col_idx - 0.5, y1=col_idx + 0.5,
        line=dict(color="black", width=3)
    )

    fig.update_layout(title_x=0.5, margin=dict(t=50, b=20))
    return fig

if __name__ == "__main__":
    app.run(debug=True)


<IPython.core.display.Javascript object>